In [104]:
import numpy as np
import matlab.engine
import math

mle = matlab.engine.start_matlab('-desktop')

In [92]:
#how to call
tunableParams = {
    'current_height': 4700.0,
    'current_vel':260.0,
    'current_pitch':0.0,
    'extension_length':0.0
}
results = mle.sim_the_model('Stoptime',STOP_TIME,"StartTime",START_TIME,'TunableParameters',tunableParams)

mle.quit()

In [93]:
def getApogee(result_matlab):
    velocity_data = result_matlab['velocity']['Data']
    velocity_time = result_matlab['velocity']['Time']
    height_data = result_matlab['height']['Data']
    height_time = result_matlab['height']['Time']

    #find time when apogee hit
    apogee_index = -1
    for i in range(0,len(velocity_data)):
        if (velocity_data[i][0]<=0 and i>1):
            apogee_index = i
            break
            
    if (apogee_index != -1):
        #find time when apogee hit
        apogee_time = velocity_time[apogee_index][0]
        return height_data[apogee_index][0]
    else:
        return -1 #INVALID APOGEE
    

In [94]:
print(getApogee(results))

7305.876945744813


In [71]:
#No airbrake deployment
#max height: 7092m
#height at burn out: 1500m
#vel at burnout: 620m/s


vel: 260 -> 0
height: 4700 -> 7092
extension: 0.0 -> 0.025 (lookup table in int percentage)

(261 x 241) lookup table

_ h   4700 4710 4720 ... 7100
v
260
259
258
.
.
.
0

mini version: (27 x 25)
____  h   4700 4800 4900 ... 7100
v
260
250
240
230
.
.
0

In [109]:
MAX_EXTENSION = 0.025
ERROR_THRESHOLD = 1
TARGET_APOG = 6850
STOP_TIME = 34
START_TIME = 5

In [110]:
def get_extension(curr_vel, curr_alt,target):
    #find apogee when no extension yet
    params = {
        'current_height': curr_alt,
        'current_vel':curr_vel,
        'current_pitch':0.0,
        'extension_length':0.0
    }
    result = mle.sim_the_model('Stoptime',STOP_TIME,"StartTime", START_TIME,'TunableParameters',params)
    curr_apog = getApogee(result)
    print("curr apogee w no extension = " + str(curr_apog))
    
    #already undershooting
    if curr_apog<target:
        return 0

    params['extension_length']= MAX_EXTENSION
    result = mle.sim_the_model('Stoptime',STOP_TIME,"StartTime", START_TIME,'TunableParameters',params)
    curr_apog = getApogee(result)
    print("curr apogee w 100% extension = " + str(curr_apog))

    #already undershooting
    if curr_apog>target:
        return 100

    #find best extension w binary search
    low = 0
    hi = 100
    while (low<hi):
        mid = (low+hi)/2
        params['extension_length']= (mid/100.0)*MAX_EXTENSION
        result = mle.sim_the_model('Stoptime',STOP_TIME,"StartTime", START_TIME,'TunableParameters',params)
        curr_apog = getApogee(result)
        if (abs(curr_apog-target)<=ERROR_THRESHOLD):
            return math.floor(mid)
        elif (curr_apog<target):
            #smaller extension
            hi = mid
        else:
            low=mid
    return 0

In [111]:
LUT = [[0]*25 for _ in range(27)]

In [114]:
#try first entry: height = 4700, vel = 260
%timeit print(get_extension(260.0,4700.0,TARGET_APOG))

curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
curr apogee w no extension = 7305.876945744813
curr apogee w 100% extension = 7304.200383381849
100
479 ms ± 84.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
mle.quit()

479 ms per call min
30129 s
8.4 hrs to generate whole table :( fun